# Using PULP

In [45]:
from pulp import *
#pulp.pulpTestAll()


In [46]:
# Create the model for the problem
prob = LpProblem("MaxProfit",LpMaximize)

In [47]:
# The 2 variables x1, x2 have a lower limit of zero
x1=LpVariable("x1",150, None)
x2=LpVariable("x2",100, None)


In [48]:
# The objective function
prob += 3500.0 *x1 + 5000.0 *x2, "Objective"

In [49]:
# The three constraints are
prob += 20.0*x1 + 40.0*x2 <= 12000.0, "Constraint 1"
prob += 25.0*x1 + 10.0*x2 <= 10000.0, "Constraint 2"
prob += 45.0*x1 + 50.0*x2 >= 14000.0, "Constraint 3"
prob += x1 <= 500.0, "Constraint 4"
prob += x2 <= 200.0, "Constraint 5"

In [50]:
# Write the problem data to an .lp file
prob.writeLP("MaxProfit.lp")


## Simple Solve

In [51]:
# Solve the optimization problem using the specified Solver
# solve the problem
#status = prob.solve(GLPK(msg=0))
#print(LpStatus[status])
 
# print the results x1 = 20, x2 = 60
#print(value(x1))
#print(value(x2))


## Solve with Sensitivity Report (GLPK Solver)

In [52]:
status=prob.solve(GLPK(options=["--ranges","MaxProfit.sen"]))
#glpsol --cpxlp MaxProfit.lp --ranges Maxprofit.sen
print(LpStatus[status])
print(value(x1))
print(value(x2))

Optimal
350.0
125.0


In [53]:
33 # Print the status of the solution
print("Status:", LpStatus[prob.status])
# Print each of the variables with it’s resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
# Print the optimised value of the objective function
print("Objective", value(prob.objective))

Status: Optimal
x1 = 350.0
x2 = 125.0
Objective 1850000.0


## Display sensitivity Report

In [ ]:
%load Maxprofit.sen

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  Objective = 1850000 (MAXimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 Constraint_1 NU   12000.00000        .               -Inf    11200.00000    -112.50000      1.76e+06 x2
                                         112.50000   12000.00000    28000.00000          +Inf      3.65e+06 x1

     2 Constraint_2 NU   10000.00000        .               -Inf     6000.00000     -50.00000      1.65e+06 x1
                                          50.00000   10000.00000    11000.00000          +Inf       1.9e+06 x2

     3 Constraint_3 BS   22000.00000   -8000.00000   14000.00000    18000.00000     -50.00000  750000.00000 Constraint_2
                                            .               +Inf    22000.00000          +Inf          +Inf

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   2

Problem:    
Objective:  Objective = 1850000 (MAXimum)

   No. Column name  St      Activity      Obj coef   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 x1           BS     350.00000    3500.00000     150.00000      -50.00000    2500.00000       1.5e+06 Constraint_2
                                            .               +Inf      360.00000   12500.00000         5e+06 Constraint_1

     2 x2           BS     125.00000    5000.00000     100.00000     -125.00000    1400.00000       1.4e+06 Constraint_1
                                            .               +Inf      225.00000    7000.00000       2.1e+06 Constraint_2

End of report


# Solve as equations

In [44]:
# initialize the model
manviMaxProfit = LpProblem("ManviMaxProfit", LpMaximize)
#List of decision variables
vehicles = ['car', 'truck']
# create a dictionary of pulp variables with keys from ingredients
# the default lower bound is -inf
x = pulp.LpVariable.dict('x_%s', vehicles, lowBound = 0)

# Objective function

profit = [3500.0, 5000.0]
cost = dict(zip(vehicles, profit))

manviMaxProfit += sum([cost[i] * x[i] for i in vehicles]), "Objective"

# Constraints
const1 = dict(zip(vehicles, [20, 40]))
const2 = dict(zip(vehicles, [25, 10]))
const3 = dict(zip(vehicles, [45, 50]))

manviMaxProfit += sum([const1[i] * x[i] for i in vehicles]) <= 12000, "Constraint 1"
manviMaxProfit += sum([const2[i] * x[i] for i in vehicles]) <= 10000, "Constraint 2"
manviMaxProfit += sum([const3[i] * x[i] for i in vehicles]) >= 14000, "Constraint 3"

mincnt = dict(zip(vehicles, [150, 100]))

for i in vehicles:
    manviMaxProfit += x[i]  >= mincnt[i]

maxcnt = dict(zip(vehicles, [500, 200]))
for i in vehicles:
    manviMaxProfit += x[i]  <= maxcnt[i]


manviMaxProfit.writeLP("manviMaxProfit.lp")
    
status=manviMaxProfit.solve(GLPK(options=["--ranges","manviMaxProfit.sen"]))
print(status)
#print the result
for vehicle in vehicles:
    print(' {} :: {} ::'.format(vehicle,
    x[vehicle].value()))
    
print("Objective", value(manviMaxProfit.objective))

1
 car :: 350.0 ::
 truck :: 125.0 ::
Objective 1850000.0


In [ ]:
# %load manviMaxProfit.sen
GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  Objective = 1850000 (MAXimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 Constraint_1 NU   12000.00000        .               -Inf    11200.00000    -112.50000      1.76e+06 _C2
                                         112.50000   12000.00000    14400.00000          +Inf      2.12e+06 _C4

     2 Constraint_2 NU   10000.00000        .               -Inf     7000.00000     -50.00000       1.7e+06 _C4
                                          50.00000   10000.00000    11000.00000          +Inf       1.9e+06 _C2

     3 Constraint_3 BS   22000.00000   -8000.00000   14000.00000    19000.00000     -50.00000  750000.00000 Constraint_2
                                            .               +Inf    22000.00000          +Inf          +Inf

     4 _C1          BS     350.00000    -200.00000     150.00000      200.00000   -1000.00000       1.5e+06 Constraint_2
                                            .               +Inf      360.00000    9000.00000         5e+06 Constraint_1

     5 _C2          BS     125.00000     -25.00000     100.00000         .        -3600.00000       1.4e+06 Constraint_1
                                            .               +Inf      200.00000    2000.00000       2.1e+06 Constraint_2

     6 _C3          BS     350.00000     150.00000          -Inf      200.00000   -1000.00000       1.5e+06 Constraint_2
                                            .          500.00000      360.00000    9000.00000         5e+06 Constraint_1

     7 _C4          BS     125.00000      75.00000          -Inf      100.00000   -3600.00000       1.4e+06 Constraint_1
                                            .          200.00000      225.00000    2000.00000       2.1e+06 Constraint_2

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   2

Problem:    
Objective:  Objective = 1850000 (MAXimum)

   No. Column name  St      Activity      Obj coef   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 x_car        BS     350.00000    3500.00000        .           200.00000    2500.00000       1.5e+06 Constraint_2
                                            .               +Inf      360.00000   12500.00000         5e+06 Constraint_1

     2 x_truck      BS     125.00000    5000.00000        .           100.00000    1400.00000       1.4e+06 Constraint_1
                                            .               +Inf      200.00000    7000.00000       2.1e+06 Constraint_2

End of report


#### Solve with Gurobi (Pending)

#### Solve with CPLEX (Pending)

## New Product (Van) introduced to mix

In [64]:
# initialize the model
manviMaxProfit = LpProblem("ManviMaxProfit", LpMaximize)
#List of decision variables
vehicles = ['car', 'truck', 'van']
# create a dictionary of pulp variables with keys from ingredients
# the default lower bound is -inf
x = pulp.LpVariable.dict('x_%s', vehicles, lowBound = 0)

# Objective function

profit = [3500.0, 5000.0, 4000.0]
cost = dict(zip(vehicles, profit))

manviMaxProfit += sum([cost[i] * x[i] for i in vehicles]), "Objective"

# Constraints
const1 = dict(zip(vehicles, [20, 40, 30]))
const2 = dict(zip(vehicles, [25, 10, 20]))
const3 = dict(zip(vehicles, [45, 50, 50]))

manviMaxProfit += sum([const1[i] * x[i] for i in vehicles]) <= 12000, "Constraint 1"
manviMaxProfit += sum([const2[i] * x[i] for i in vehicles]) <= 10000, "Constraint 2"
manviMaxProfit += sum([const3[i] * x[i] for i in vehicles]) >= 14000, "Constraint 3"

mincnt = dict(zip(vehicles, [150, 100, 0]))

for i in vehicles:
    manviMaxProfit += x[i]  >= mincnt[i]

maxcnt = dict(zip(vehicles, [500, 200, 1000]))
for i in vehicles:
    manviMaxProfit += x[i]  <= maxcnt[i]


manviMaxProfit.writeLP("manviMaxProfit2.lp")
    
status=manviMaxProfit.solve(GLPK(options=["--ranges","manviMaxProfit2.sen"]))
print(status)
#print the result
for vehicle in vehicles:
    print(' {} :: {} ::'.format(vehicle,
    x[vehicle].value()))
    
print("Objective", value(manviMaxProfit.objective))

1
 car :: 350.0 ::
 truck :: 125.0 ::
 van :: 0.0 ::
Objective 1850000.0


In [ ]:
# %load manviMaxProfit2.sen
GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  Objective = 1850000 (MAXimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 Constraint_1 NU   12000.00000        .               -Inf    11200.00000    -112.50000      1.76e+06 _C2
                                         112.50000   12000.00000    14400.00000          +Inf      2.12e+06 _C5

     2 Constraint_2 NU   10000.00000        .               -Inf     7000.00000     -50.00000       1.7e+06 _C5
                                          50.00000   10000.00000    11000.00000          +Inf       1.9e+06 _C2

     3 Constraint_3 BS   22000.00000   -8000.00000   14000.00000    19000.00000     -50.00000  750000.00000 Constraint_2
                                            .               +Inf    22000.00000          +Inf          +Inf

     4 _C1          BS     350.00000    -200.00000     150.00000      314.28571    -600.00000      1.64e+06 x_van
                                            .               +Inf      360.00000    9000.00000         5e+06 Constraint_1

     5 _C2          BS     125.00000     -25.00000     100.00000         .         -857.14286   1.74286e+06 x_van
                                            .               +Inf      200.00000    2000.00000       2.1e+06 Constraint_2

     6 _C3          BS        .             .             .              .               -Inf      1.85e+06
                                            .               +Inf       57.14286     375.00000      1.85e+06 x_van

     7 _C4          BS     350.00000     150.00000          -Inf      314.28571    -600.00000      1.64e+06 x_van
                                            .          500.00000      360.00000    9000.00000         5e+06 Constraint_1

     8 _C5          BS     125.00000      75.00000          -Inf      100.00000    -857.14286   1.74286e+06 x_van
                                            .          200.00000      225.00000    2000.00000       2.1e+06 Constraint_2

     9 _C6          BS        .         1000.00000          -Inf         .               -Inf      1.85e+06
                                            .         1000.00000       57.14286     375.00000      1.85e+06 x_van

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   2

Problem:    
Objective:  Objective = 1850000 (MAXimum)

   No. Column name  St      Activity      Obj coef   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 x_car        BS     350.00000    3500.00000        .           314.28571    2900.00000      1.64e+06 x_van
                                            .               +Inf      360.00000   12500.00000         5e+06 Constraint_1

     2 x_truck      BS     125.00000    5000.00000        .           100.00000    4142.85714   1.74286e+06 x_van
                                            .               +Inf      200.00000    7000.00000       2.1e+06 Constraint_2

     3 x_van        NL        .         4000.00000        .              .               -Inf      1.85e+06 _C3
                                        -375.00000          +Inf       57.14286    4375.00000   1.82857e+06 _C2

End of report
